In [1]:
import json

In [2]:
data = json.load(open('ucf101_01.json', 'r'))

In [3]:
mini = ['PlayingFlute', 'Biking', 'SkateBoarding', 'TableTennisShot', 'Typing', 'SkyDiving', 'TrampolineJumping', 'BrushingTeeth', 'PullUps', 'YoYo']

In [4]:
mini_data = {"labels": [], "database": {}}

In [5]:
mini_data['labels'] = mini

In [6]:
cnt = 0
for train in data['database']:
    if train.split('_')[1] in mini:
        mini_data["database"][train] = data['database'][train]

In [7]:
with open('ucf101_10_class.json', 'w') as f:
    json.dump(mini_data, f, ensure_ascii=False, indent=4)